In [1]:
import rdflib
import time
import pandas as pd
import os
import s3fs
from dotenv import load_dotenv
from utils.dcat_reader_ckan import DCATReaderCKAN

load_dotenv()  # take environment variables from .env.

True

In [2]:
import glob
import pandas as pd
import json

# Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL}, key ='HJAB2OJ1IKLQXY9UAA1O', secret = 'lWgTcFTbUX3DTcBQSpViNvg7R+yVClgY0utMFs5c', token = 'eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJhY2Nlc3NLZXkiOiJISkFCMk9KMUlLTFFYWTlVQUExTyIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sImF1ZCI6WyJtaW5pby1kYXRhbm9kZSIsIm9ueXhpYSIsImFjY291bnQiXSwiYXV0aF90aW1lIjoxNjk4MDY2NTc0LCJhenAiOiJvbnl4aWEiLCJlbWFpbCI6InF1ZW50aW4ubG9yaWRhbnRAbXVsdGkuY29vcCIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJleHAiOjE2OTg0MTk3NTUsImZhbWlseV9uYW1lIjoiTG9yaWRhbnQiLCJnaXZlbl9uYW1lIjoiUXVlbnRpbiIsImdyb3VwcyI6WyJlY29sYWItYWN0aW9uLXF1YWxpdGUiXSwiaWF0IjoxNjk4MzMzMzU1LCJpc3MiOiJodHRwczovL2F1dGgubGFiLnNzcGNsb3VkLmZyL2F1dGgvcmVhbG1zL3NzcGNsb3VkIiwianRpIjoiZjY1NzI2MTEtYWMzOC00NWZlLTgwZjEtZDZhZDI4MjdjYjZmIiwibmFtZSI6IlF1ZW50aW4gTG9yaWRhbnQiLCJwb2xpY3kiOiJzdHNvbmx5IiwicHJlZmVycmVkX3VzZXJuYW1lIjoicXVlbnRpbmxvcmlkYW50IiwicmVhbG1fYWNjZXNzIjp7InJvbGVzIjpbIm9mZmxpbmVfYWNjZXNzIiwidW1hX2F1dGhvcml6YXRpb24iLCJkZWZhdWx0LXJvbGVzLXNzcGNsb3VkIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwic2NvcGUiOiJvcGVuaWQgcHJvZmlsZSBncm91cHMgZW1haWwiLCJzZXNzaW9uUG9saWN5IjoiZXlKV1pYSnphVzl1SWpvaU1qQXhNaTB4TUMweE55SXNJbE4wWVhSbGJXVnVkQ0k2VzNzaVJXWm1aV04wSWpvaVFXeHNiM2NpTENKQlkzUnBiMjRpT2xzaWN6TTZLaUpkTENKU1pYTnZkWEpqWlNJNld5SmhjbTQ2WVhkek9uTXpPam82Y0hKdmFtVjBMV1ZqYjJ4aFlpMWhZM1JwYjI0dGNYVmhiR2wwWlNJc0ltRnlianBoZDNNNmN6TTZPanB3Y205cVpYUXRaV052YkdGaUxXRmpkR2x2YmkxeGRXRnNhWFJsTHlvaVhYMHNleUpGWm1abFkzUWlPaUpCYkd4dmR5SXNJa0ZqZEdsdmJpSTZXeUp6TXpwTWFYTjBRblZqYTJWMElsMHNJbEpsYzI5MWNtTmxJanBiSW1GeWJqcGhkM002Y3pNNk9qb3FJbDBzSWtOdmJtUnBkR2x2YmlJNmV5SlRkSEpwYm1kTWFXdGxJanA3SW5Nek9uQnlaV1pwZUNJNkltUnBabVoxYzJsdmJpOHFJbjE5ZlN4N0lrVm1abVZqZENJNklrRnNiRzkzSWl3aVFXTjBhVzl1SWpwYkluTXpPa2RsZEU5aWFtVmpkQ0pkTENKU1pYTnZkWEpqWlNJNld5SmhjbTQ2WVhkek9uTXpPam82S2k5a2FXWm1kWE5wYjI0dktpSmRmVjE5Iiwic2Vzc2lvbl9zdGF0ZSI6IjYwNmU0MDFkLWE1NDktNDkyZS1hNjQ2LWZlYTI5OWFiOTMxOSIsInNpZCI6IjYwNmU0MDFkLWE1NDktNDkyZS1hNjQ2LWZlYTI5OWFiOTMxOSIsInN1YiI6Ijg1NjQzNjFmLWQ3MTctNDhhYy04YTA2LTdlZjY5ZTlmZTRiMCIsInR5cCI6IkJlYXJlciJ9.phmh-NLxDB1AMGzDrR9Ge-Yhp4mJwkFjpG2VznkaaIVfHJE9WJybjti6iZ9y30bDOfgO9ubAiORqDY9SxL2Ngg')

data = []
iteration = 0
BUCKET = "projet-ecolab-action-qualite"
FILE_KEY_S3 = "DUMP_INTEGRATION_JSON/integration_page_107_2023-10-14.json"
FILE_PATH_S3 = BUCKET + "/" + FILE_KEY_S3


print(len(fs.ls(BUCKET + '/DUMP_INTEGRATION_JSON')))
      
for graph in fs.ls(BUCKET + '/DUMP_INTEGRATION_JSON')[0:10]:
    if '.json' in graph:
        with fs.open(graph, mode="rb") as file_in:
            read_data = file_in.read()
            page_data = DCATReaderCKAN(read_data).get_data()
            print("Fichier: ", graph, "- Nombre de datasets : ", len(page_data))
            data.append(
                page_data
            )
            iteration += 1

print("Nombre de pages traitées : ", iteration)
data = pd.concat(data)


239
Fichier:  projet-ecolab-action-qualite/DUMP_INTEGRATION_JSON/integration_page_100_2023-10-14.json - Nombre de datasets :  100
Fichier:  projet-ecolab-action-qualite/DUMP_INTEGRATION_JSON/integration_page_101_2023-10-14.json - Nombre de datasets :  100
Fichier:  projet-ecolab-action-qualite/DUMP_INTEGRATION_JSON/integration_page_102_2023-10-14.json - Nombre de datasets :  99
Fichier:  projet-ecolab-action-qualite/DUMP_INTEGRATION_JSON/integration_page_103_2023-10-14.json - Nombre de datasets :  100
Fichier:  projet-ecolab-action-qualite/DUMP_INTEGRATION_JSON/integration_page_104_2023-10-14.json - Nombre de datasets :  100
Fichier:  projet-ecolab-action-qualite/DUMP_INTEGRATION_JSON/integration_page_105_2023-10-14.json - Nombre de datasets :  100
Fichier:  projet-ecolab-action-qualite/DUMP_INTEGRATION_JSON/integration_page_106_2023-10-14.json - Nombre de datasets :  100
Fichier:  projet-ecolab-action-qualite/DUMP_INTEGRATION_JSON/integration_page_107_2023-10-14.json - Nombre de datas

In [7]:
from sqlalchemy import create_engine
engine = create_engine(f"postgresql+psycopg2://{os.environ['POSTGRES_USER']}:{os.environ['POSTGRES_PASSWORD']}@{os.environ['POSTGRES_URL']}:{os.environ['POSTGRES_PORT']}/{os.environ['POSTGRES_DB']}")

with engine.begin() as connection:
    data.to_sql(name='datasets', con=connection, if_exists='replace', index=False)
